# Example usage notebook of the DD-GAN repo

In [1]:
import time
import numpy as np
import sklearn.preprocessing
import tensorflow as tf

After installation of the ddgan package run the following line to import it

In [2]:
from ddgan import GAN
from ddgan import *

In [3]:
kwargs = {
    "nsteps" : 5,
    "ndims" : 5,
    "lmbda" : 10,
    "n_critic" : 5,
    "batches" : 10,
    "batch_size" : 20,
    "seed" : 143
}

gan = GAN(**kwargs)
gan.setup()
set_seed(gan.seed)

ndims_latent_input = 5

#csv_data = np.load('../data/processed/pod_coeffs_field_Velocity_onegrid.npy', )
csv_data = np.loadtxt('../data/processed/POD_coeffs_1_204_orig.csv', delimiter=',')
csv_data = np.float32(csv_data)
#csv_data = csv_data[0,:,:]ææææææææææáððððáþÞÆ
                    

print('type and shape (nPOD by nTrain) of POD coeffs from csv file', type(csv_data), csv_data.shape, csv_data.dtype)


looking for previous saved models
making new generator and critic
type and shape (nPOD by nTrain) of POD coeffs from csv file <class 'numpy.ndarray'> (5, 204) float32


In [4]:
nTrain = csv_data.shape[1]
nPOD = csv_data.shape[0]

csv_data = csv_data.T # nTrain by nPOD


In [5]:
scaling = sklearn.preprocessing.MinMaxScaler(feature_range=[-1,1])
csv_data = scaling.fit_transform(csv_data)

for icol in range(csv_data.shape[1]):
    print('min and max of col, ', icol ,' of csv_data:', np.min(csv_data[:,icol]), np.max(csv_data[:,icol]) )

min and max of col,  0  of csv_data: -1.0 1.0
min and max of col,  1  of csv_data: -1.0 1.0000001
min and max of col,  2  of csv_data: -1.0 1.0
min and max of col,  3  of csv_data: -1.0 1.0
min and max of col,  4  of csv_data: -1.0 1.0


In [6]:
nsteps = 5
t_begin = 0
t_end = nTrain - nsteps + 1
training_data = np.zeros((t_end, nPOD * nsteps), dtype=np.float32) # nTrain by nsteps*nPOD # 'float32' or np.float32

input_to_GAN = tf.random.normal([training_data.shape[0], ndims_latent_input])
input_to_GAN = input_to_GAN.numpy()

for step in range(nsteps):
    training_data[:,step*nPOD:(step+1)*nPOD] = csv_data[t_begin+step : t_end+step,:]
print('Shape of training data for the GAN', training_data.shape, training_data.dtype)

Shape of training data for the GAN (200, 25) float32


In [7]:
t0 = time.time()
gan.learn_hypersurface_from_POD_coeffs(nPOD, input_to_GAN, training_data, ndims_latent_input)
t_train = time.time() - t0

beginning training
epoch: 0
[<tf.Variable 'dense/kernel:0' shape=(5, 5) dtype=float32, numpy=
array([[-0.6327522 , -0.50542   ,  0.4299817 ,  0.29948306, -0.5596052 ],
       [-0.699732  , -0.23958492, -0.27966952,  0.5152048 , -0.38847622],
       [-0.7379004 , -0.07778484, -0.2784693 , -0.06087399,  0.6034943 ],
       [ 0.26544726,  0.39098716,  0.4104302 ,  0.61889076, -0.42484418],
       [-0.00462967, -0.12501293, -0.23175311, -0.42557347,  0.7655997 ]],
      dtype=float32)>, <tf.Variable 'dense/bias:0' shape=(5,) dtype=float32, numpy=array([0., 0., 0., 0., 0.], dtype=float32)>]
gen loss -0.0009700097 d loss 2.3174634 w_loss 0.24425566
epoch: 1
[<tf.Variable 'dense/kernel:0' shape=(5, 5) dtype=float32, numpy=
array([[-0.63208133, -0.50564057,  0.4306676 ,  0.29973158, -0.55968606],
       [-0.6994412 , -0.2396412 , -0.2793178 ,  0.51527137, -0.38895768],
       [-0.73864734, -0.07703961, -0.27811903, -0.06167014,  0.6033211 ],
       [ 0.26564205,  0.39115995,  0.40996096,  0.61

gen loss 0.25505334 d loss 1.4764977 w_loss 0.6393962
epoch: 12
[<tf.Variable 'dense/kernel:0' shape=(5, 5) dtype=float32, numpy=
array([[-0.63147104, -0.50464517,  0.43192157,  0.30063018, -0.5597366 ],
       [-0.6998341 , -0.24256   , -0.28060752,  0.51833445, -0.3906113 ],
       [-0.7401193 , -0.07600757, -0.2764669 , -0.06207204,  0.6040662 ],
       [ 0.26280367,  0.3908023 ,  0.40869087,  0.61558896, -0.4240062 ],
       [-0.00547732, -0.12264955, -0.23200198, -0.42496184,  0.7641518 ]],
      dtype=float32)>, <tf.Variable 'dense/bias:0' shape=(5,) dtype=float32, numpy=
array([-0.00079348,  0.00108555, -0.0013911 ,  0.00087117, -0.00055054],
      dtype=float32)>]


KeyboardInterrupt: 

In [ ]:
gan.generator.layers[0].get_weights()